# Processing REDUCEDHEATARB data for heat distribution system model check

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

# Files needed as input
rhc_preprocessed_poperties_file='rhc_preprocessed_properties.parquet'
home_data_file_path = "home_data.parquet"
boiler_returntemp_load_efficiency_file_path = "boiler_returntemp_load_efficiency.parquet"
boiler_returntemp_efficiency_file_path = "boiler_returntemp_efficiency.parquet"

# Files written as output
rhc_heat_dist_preprocessed_poperties_file='rhc_heat_dist_preprocessed_properties.parquet'

%load_ext autoreload

%matplotlib inline
%matplotlib widget

import sys
sys.path.append('../analysis')
from nfh_utils import *

## Reading preprocessed interpolated properties from a parquet file

In [ ]:
%%time


# Attempt to read the Parquet file
try:
    df_prep = pd.read_parquet(
        rhc_preprocessed_poperties_file, 
        engine='pyarrow',
        dtype_backend='numpy_nullable'
        )
    print("File was successfully read without specifying compression codec.")
except Exception as e:
    print(f"Error reading file: {e}")

#sorting the DataFrame index is needed to get good performance on certain filters
#this guarding code to check whether DataFrames are properly sorted
if not df_prep.index.is_monotonic_increasing:
    print('df needed index sorting')
    df_prep = df_prep.sort_index()  

In [ ]:
print(f"df_prep.count().sum(): {df_prep.count().sum():_}")

In [ ]:
df_prep.info()

In [ ]:
print("[\n'","', \n'".join(sorted(df_prep.columns)),"'\n]")

## Filtering values immediately relevant for heat distribution system modelling

In [ ]:
heat_distribution_props = [
    'batch_import_KNMI_ghi__W_m_2', 
    'batch_import_KNMI_temp_out__degC', 
    'batch_import_KNMI_wind__m_s_1', 
    'device_p1-reader_g_use_hhv__W', 
    'batch_import_remeha_boiler_status_burning_ch__bool',
    'batch_import_remeha_gas_valve_closed__bool', 
    'batch_import_remeha_gas_valve_open__bool', 
    'batch_import_remeha_g_use_ch_lhv__W',
    'batch_import_remeha_temp_set__degC', 
    'batch_import_remeha_temp_in__degC', 
    'device_living_room_calibrated_temp_in__degC', 
    'batch_import_remeha_temp_sup__degC', 
    'batch_import_remeha_temp_ret__degC' 
]

In [ ]:
df_heat_dist = df_prep[heat_distribution_props].copy()

In [ ]:
print(f"df_heat_dist.count().sum(): {df_heat_dist.count().sum():_}")

In [ ]:
df_heat_dist.info()

## Calculating additional values relevant for heat distribution system modelling

In [ ]:
%%time

# Filter out rows where either of the columns has NaN
valid_rows = df_heat_dist[['batch_import_remeha_temp_sup__degC', 'batch_import_remeha_temp_ret__degC']].notna().all(axis=1)

# Calculate the mean only for valid rows
df_heat_dist.loc[valid_rows, 'calculated_temp_rad__degC'] = df_heat_dist.loc[valid_rows, ['batch_import_remeha_temp_sup__degC', 'batch_import_remeha_temp_ret__degC']].mean(axis=1)

## Reading boiler data

In [ ]:
%%time
# Attempt to read the Parquet file
try:
    df_homes = pd.read_parquet(
        home_data_file_path, 
        engine='pyarrow',
        dtype_backend='numpy_nullable'
        )
    print("File was successfully read without specifying compression codec.")
except Exception as e:
    print(f"Error reading file: {e}")

In [ ]:
df_homes

In [ ]:
%%time
# Attempt to read the Parquet file
try:
    df_boiler_efficiency = pd.read_parquet(
        boiler_returntemp_efficiency_file_path, 
        engine='pyarrow',
        dtype_backend='numpy_nullable'
        )
    print("File was successfully read without specifying compression codec.")
except Exception as e:
    print(f"Error reading file: {e}")

In [ ]:
df_boiler_efficiency

## Calculating heat production

In [ ]:
# # Calculate Q_gain_ch__W based on national average efficiency
# df_heat_dist['calculated_Q_gain_ch__W'] = df_heat_dist['batch_import_remeha_g_use_ch_lhv__W'] *  g_groningen_hhv___MJ_m_3 /  g_groningen_lhv___MJ_m_3 * eta_ch_nl_avg_hhv__J0

In [ ]:
%%time
# Round return temperatures to whole degrees
df_heat_dist['rounded_temp_ret__degC'] = df_heat_dist['batch_import_remeha_temp_ret__degC'].round()


# Extract id and rounded_temp_ret__degC values
id_values = df_heat_dist.index.get_level_values('id')
temp_ret_values = df_heat_dist['rounded_temp_ret__degC']

# Lookup efficiency vectorized
def lookup_efficiency_vectorized(id_vals, temp_vals, df_efficiency, df_homes):
    brand_models = df_homes.loc[id_vals, 'brand_model'].values
    efficiency_values = np.empty(len(brand_models))
    
    for i, (model, temp) in enumerate(zip(brand_models, temp_vals)):
        try:
            efficiency_values[i] = df_efficiency.loc[(model, temp), 'eta_ch_hhv__W0']
        except KeyError:
            efficiency_values[i] = np.nan  # or some default value
    
    return efficiency_values


# Apply the vectorized lookup
df_heat_dist['eta_ch_hhv__W0'] = lookup_efficiency_vectorized(id_values, temp_ret_values, df_boiler_efficiency, df_homes)

In [ ]:
# # Merge df_heat_dist with df_homes to get the brand_model for each id
# df_heat_dist = df_heat_dist.reset_index().merge(df_homes, on='id').set_index(['id', 'timestamp'])

# # Round return temperatures to whole degrees and convert to nullable integer type
# df_heat_dist['rounded_temp_ret__degC'] = df_heat_dist['batch_import_remeha_temp_ret__degC'].round().astype('Int64')

# # Merge with df_boiler_efficiency to get the efficiency values
# df_heat_dist = df_heat_dist.reset_index().merge(
#     df_boiler_efficiency.reset_index(),
#     left_on=['brand_model', 'rounded_temp_ret__degC'],
#     right_on=['boiler_model', 'temp_ret__degC'],
#     how='left'
# ).set_index(['id', 'timestamp'])

# # Perform the calculation
# df_heat_dist['calculated_Q_gain_ch__W'] = (
#     df_heat_dist['batch_import_remeha_g_use_ch_lhv__W'] *
#     g_groningen_hhv___MJ_m_3 / g_groningen_lhv___MJ_m_3 *
#     df_heat_dist['eta_ch_hhv__W0']
# )

In [ ]:
df_heat_dist['eta_ch_hhv__W0'].mean()

In [ ]:
df_heat_dist['calculated_Q_gain_ch__W'] = df_heat_dist['batch_import_remeha_g_use_ch_lhv__W'] *  g_groningen_hhv___MJ_m_3 /  g_groningen_lhv___MJ_m_3 * df_heat_dist['eta_ch_hhv__W0']

In [ ]:
df_heat_dist = df_heat_dist[['batch_import_KNMI_ghi__W_m_2',
                             'batch_import_KNMI_temp_out__degC',
                             'batch_import_KNMI_wind__m_s_1',
                             'device_p1-reader_g_use_hhv__W',
                             'batch_import_remeha_boiler_status_burning_ch__bool',
                             'batch_import_remeha_gas_valve_closed__bool',
                             'batch_import_remeha_gas_valve_open__bool',
                             'batch_import_remeha_g_use_ch_lhv__W',
                             'batch_import_remeha_temp_set__degC',
                             'batch_import_remeha_temp_in__degC',
                             'device_living_room_calibrated_temp_in__degC',
                             'batch_import_remeha_temp_sup__degC',
                             'batch_import_remeha_temp_ret__degC',
                             'calculated_temp_rad__degC',
                             'calculated_Q_gain_ch__W',
                             'rounded_temp_ret__degC',
                             'eta_ch_hhv__W0']]

In [ ]:
list(df_heat_dist.columns)

In [ ]:
df_heat_dist.describe().T

In [ ]:
df_heat_dist

## Writing results to parquet file

In [ ]:
%%time 
df_heat_dist.to_parquet(rhc_heat_dist_preprocessed_poperties_file, index=True, engine='pyarrow')

## Writing results to multiple zipped CSV files

In [ ]:
# %%time 
# # uncomment this entire block of code to enable it 
# for home_id in tqdm(df_heat_dist.index.get_level_values('id').unique()):
#     df_heat_dist.xs(home_id, drop_level=False).to_csv(
#         f'{home_id}_preprocessed_properties.zip',
#         encoding='utf-8',
#         compression= dict(method='zip',
#                           archive_name=f'{home_id}_preprocessed_properties.csv'),
#         date_format='%Y-%m-%dT%H:%M:%S%z'
#     )
